# Deep Research Agents クックブック

このクックブックでは、OpenAI Deep Research APIとOpenAI [Agents SDK](https://openai.github.io/openai-agents-python/)を使用してAgenticリサーチワークフローを構築する方法を説明します。これは[基礎クックブック](https://cookbook.openai.com/examples/deep_research_api/introduction_to_deep_research_api)の続編です。まだその内容に慣れ親しんでいない場合は、まずそちらをご覧になることをお勧めします。

シングルエージェントとマルチエージェントパイプラインの調整、出力品質を最大化するためのユーザークエリの拡充、リサーチ進捗のストリーミング、ウェブ検索と[内部ファイル検索のためのMCP](https://cookbook.openai.com/examples/deep_research_api/how_to_build_a_deep_research_mcp_server/readme)の統合、そして堅牢なリサーチアプリケーションの設計について学習します。

計画、統合、ツール使用、または多段階推論が必要なタスクにはDeep Research Agentsの使用を検討してください。些細な事実確認、単純なQ&A、短文チャットには Deep Research を使用しないでください。通常のopenai.responsesAPIの方が高速で安価です。

### 前提条件
* OpenAI API キー（環境変数として OPENAI_API_KEY を設定）
* Agents SDK と OpenAI Python SDK

### セットアップ
*依存関係のインストール*

In [ ]:
%pip install --upgrade "openai>=1.88" "openai-agents>=0.0.19"

### ライブラリのインポートとクライアントの設定

**ゼロデータ保持**

以下の`os.environ`設定により、データ保持を無効にします。これにより、企業はDeep Researchでゼロデータ保持環境で運用することができます。データ保持があなたにとって積極的な制約で_ない_場合は、エージェントワークフローの自動追跡機能や、評価やファインチューニングなどの他のプラットフォームツールとの深い統合を利用できるよう、有効のままにしておくことを検討してください。

In [2]:
import os
from agents import Agent, Runner, WebSearchTool, RunConfig, set_default_openai_client, HostedMCPTool
from typing import List, Dict, Optional
from pydantic import BaseModel
from openai import AsyncOpenAI

# Use env var for API key and set a long timeout
client = AsyncOpenAI(api_key="", timeout=600.0)
set_default_openai_client(client)
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "1" # Disable tracing for Zero Data Retention (ZDR) Organizations

### 基本的なディープリサーチエージェント

基本的なリサーチエージェントは、o4-mini-deep-research-alphaモデルを使用してディープリサーチを実行します。このエージェントは、パブリックインターネットへのネイティブWebSearch機能を持ち、その調査結果を直接ノートブックにストリーミングして返します。この場合、フルのo3ディープリサーチモデルよりも高速でありながら、十分な知能を持つ`o4-mini-deep-research-alpha`モデルを使用しています。

**学習目標:**

この後、単一エージェントのリサーチタスクを実行し、その進捗をストリーミングできるようになります。

In [ ]:
# Define the research agent
research_agent = Agent(
    name="Research Agent",
    model="o4-mini-deep-research-2025-06-26",
    tools=[WebSearchTool()],
    instructions="You perform deep empirical research based on the user's question."
)

# Async function to run the research and print streaming progress
async def basic_research(query):
    print(f"Researching: {query}")
    result_stream = Runner.run_streamed(
        research_agent,
        query
    )

    async for ev in result_stream.stream_events():
        if ev.type == "agent_updated_stream_event":
            print(f"\n--- switched to agent: {ev.new_agent.name} ---")
            print(f"\n--- RESEARCHING ---")
        elif (
            ev.type == "raw_response_event"
            and hasattr(ev.data, "item")
            and hasattr(ev.data.item, "action")
        ):
            action = ev.data.item.action or {}
            if action.get("type") == "search":
                print(f"[Web search] query={action.get('query')!r}")

    # streaming is complete → final_output is now populated
    return result_stream.final_output

# Run the research and print the result
result = await basic_research("Research the economic impact of semaglutide on global healthcare systems.")
print(result)

### 明確化を伴うマルチエージェント研究

マルチエージェント深層研究

「深層研究」が生成する研究品質をさらに向上させる方法について考えてみましょう。この場合、マルチエージェントアーキテクチャを活用して、深層研究エージェントに提出する前に、ユーザーのクエリと最終的な研究レポートで期待される内容について_より多くの情報_でプロンプトを充実させています。

## サブエージェントプロンプトの充実化

サポートエージェントのプロンプトは、ユーザーの初期クエリに構造と厳密性を提供することで、最終的な研究成果の品質を向上させるために特別に設計されています。

In [5]:
# ─────────────────────────────────────────────────────────────
#  Prompts
# ─────────────────────────────────────────────────────────────

CLARIFYING_AGENT_PROMPT =  """
    If the user hasn't specifically asked for research (unlikely), ask them what research they would like you to do.

        GUIDELINES:
        1. **Be concise while gathering all necessary information** Ask 2–3 clarifying questions to gather more context for research.
        - Make sure to gather all the information needed to carry out the research task in a concise, well-structured manner. Use bullet points or numbered lists if appropriate for clarity. Don't ask for unnecessary information, or information that the user has already provided.
        2. **Maintain a Friendly and Non-Condescending Tone**
        - For example, instead of saying “I need a bit more detail on Y,” say, “Could you share more detail on Y?”
        3. **Adhere to Safety Guidelines**
        """

RESEARCH_INSTRUCTION_AGENT_PROMPT = """

        Based on the following guidelines, take the users query, and rewrite it into detailed research instructions. OUTPUT ONLY THE RESEARCH INSTRUCTIONS, NOTHING ELSE. Transfer to the research agent.

        GUIDELINES:
        1. **Maximize Specificity and Detail**
        - Include all known user preferences and explicitly list key attributes or dimensions to consider.
        - It is of utmost importance that all details from the user are included in the expanded prompt.

        2. **Fill in Unstated But Necessary Dimensions as Open-Ended**
        - If certain attributes are essential for a meaningful output but the user has not provided them, explicitly state that they are open-ended or default to “no specific constraint.”

        3. **Avoid Unwarranted Assumptions**
        - If the user has not provided a particular detail, do not invent one.
        - Instead, state the lack of specification and guide the deep research model to treat it as flexible or accept all possible options.

        4. **Use the First Person**
        - Phrase the request from the perspective of the user.

        5. **Tables**
        - If you determine that including a table will help illustrate, organize, or enhance the information in your deep research output, you must explicitly request that the deep research model provide them.
        Examples:
        - Product Comparison (Consumer): When comparing different smartphone models, request a table listing each model’s features, price, and consumer ratings side-by-side.
        - Project Tracking (Work): When outlining project deliverables, create a table showing tasks, deadlines, responsible team members, and status updates.
        - Budget Planning (Consumer): When creating a personal or household budget, request a table detailing income sources, monthly expenses, and savings goals.
        Competitor Analysis (Work): When evaluating competitor products, request a table with key metrics—such as market share, pricing, and main differentiators.

        6. **Headers and Formatting**
        - You should include the expected output format in the prompt.
        - If the user is asking for content that would be best returned in a structured format (e.g. a report, plan, etc.), ask the Deep Research model to “Format as a report with the appropriate headers and formatting that ensures clarity and structure.”

        7. **Language**
        - If the user input is in a language other than English, tell the model to respond in this language, unless the user query explicitly asks for the response in a different language.

        8. **Sources**
        - If specific sources should be prioritized, specify them in the prompt.
        - Prioritize Internal Knowledge. Only retrieve a single file once.
        - For product and travel research, prefer linking directly to official or primary websites (e.g., official brand sites, manufacturer pages, or reputable e-commerce platforms like Amazon for user reviews) rather than aggregator sites or SEO-heavy blogs.
        - For academic or scientific queries, prefer linking directly to the original paper or official journal publication rather than survey papers or secondary summaries.
        - If the query is in a specific language, prioritize sources published in that language.

        IMPORTANT: Ensure that the complete payload to this function is valid JSON
        IMPORTANT: SPECIFY REQUIRED OUTPUT LANGUAGE IN THE PROMPT
        """

# 4エージェント深層研究パイプライン

1. **トリアージエージェント**  
   - ユーザーのクエリを検査  
   - コンテキストが不足している場合は明確化エージェントにルーティング、そうでなければ指示エージェントにルーティング  

2. **明確化エージェント**  
   - フォローアップ質問を行う  
   - ユーザー（またはモック）の回答を待機  

3. **指示構築エージェント**  
   - 拡充された入力を正確な研究概要に変換  

4. **研究エージェント** (`o3-deep-research`)  
   - `WebSearchTool`を使用してWebスケールの実証研究を実行
   - MCPを使用して内部ナレッジストアに対する検索を実行し、関連するドキュメントがある場合、エージェントはそれらの関連スニペットを参考資料に組み込む   
   - 透明性のために中間イベントをストリーミング
   - 最終的な研究成果物を出力（後でパースする）

![../../images/agents_dr.png](../../../images/agent_dr.png)

MCPサーバーが_どのように_構築されるかについてのより詳しい洞察については、[こちらのリソースを参照してください。](https://cookbook.openai.com/examples/deep_research_api/how_to_build_a_deep_research_mcp_server/readme )

In [6]:
# ─────────────────────────────────────────────────────────────
# Structured outputs (needed only for Clarifying agent)
# ─────────────────────────────────────────────────────────────
class Clarifications(BaseModel):
    questions: List[str]

# ─────────────────────────────────────────────────────────────
# Agents
# ─────────────────────────────────────────────────────────────
research_agent = Agent(
    name="Research Agent",
    model="o3-deep-research-2025-06-26",
    instructions="Perform deep empirical research based on the user's instructions.",
    tools=[WebSearchTool(),
           HostedMCPTool(
            tool_config={
                "type": "mcp",
                "server_label": "file_search",
                "server_url": "https://<url>/sse",
                "require_approval": "never",
            }
        )
    ]
)

instruction_agent = Agent(
    name="Research Instruction Agent",
    model="gpt-4o-mini",
    instructions=RESEARCH_INSTRUCTION_AGENT_PROMPT,
    handoffs=[research_agent],
)

clarifying_agent = Agent(
    name="Clarifying Questions Agent",
    model="gpt-4o-mini",
    instructions=CLARIFYING_AGENT_PROMPT,
    output_type=Clarifications,
    handoffs=[instruction_agent],
)

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "Decide whether clarifications are required.\n"
        "• If yes → call transfer_to_clarifying_questions_agent\n"
        "• If no  → call transfer_to_research_instruction_agent\n"
        "Return exactly ONE function-call."
    ),
    handoffs=[clarifying_agent, instruction_agent],
)


# ─────────────────────────────────────────────────────────────
#  Auto-clarify helper
# ─────────────────────────────────────────────────────────────
async def basic_research(
    query: str,
    mock_answers: Optional[Dict[str, str]] = None,
    verbose: bool = False,
):
    stream = Runner.run_streamed(
        triage_agent,
        query,
        run_config=RunConfig(tracing_disabled=True),
    )

    async for ev in stream.stream_events():
        if isinstance(getattr(ev, "item", None), Clarifications):
            reply = []
            for q in ev.item.questions:
                ans = (mock_answers or {}).get(q, "No preference.")
                reply.append(f"**{q}**\n{ans}")
            stream.send_user_message("\n\n".join(reply))
            continue
        if verbose:
            print(ev)

    #return stream.final_output
    return stream

# ─────────────────────────────────────────────────────────────
#  Example run
# ─────────────────────────────────────────────────────────────
result = await basic_research(
    "Research the economic impact of semaglutide on global healthcare systems.",
    mock_answers={},   # or provide canned answers
)

## エージェント相互作用フロー

Agent SDKトレースを通じてネイティブに提供されていますが、ツール呼び出しを含む人間が読みやすい高レベルのエージェント相互作用フローを印刷したい場合があります。`print_agent_interaction`を実行して、以下を含む簡略化された読みやすいエージェントステップのシーケンスを取得してください：エージェント名、イベントの種類（ハンドオフ、ツール呼び出し、メッセージ出力）、簡潔なツール呼び出し情報（ツール名と引数）。

In [ ]:
import json

def parse_agent_interaction_flow(stream):
    print("=== Agent Interaction Flow ===")
    count = 1

    for item in stream.new_items:
        # Agent name, fallback if missing
        agent_name = getattr(item.agent, "name", "Unknown Agent") if hasattr(item, "agent") else "Unknown Agent"

        if item.type == "handoff_call_item":
            func_name = getattr(item.raw_item, "name", "Unknown Function")
            print(f"{count}. [{agent_name}] → Handoff Call: {func_name}")
            count += 1

        elif item.type == "handoff_output_item":
            print(f"{count}. [{agent_name}] → Handoff Output")
            count += 1

        elif item.type == "mcp_list_tools_item":
            print(f"{count}. [{agent_name}] → mcp_list_tools_item")
            count += 1

        elif item.type == "reasoning_item":
            print(f"{count}. [{agent_name}] → Reasoning step")
            count += 1

        elif item.type == "tool_call_item":
            tool_name = getattr(item.raw_item, "name", None)

            # Skip tool call if tool_name is missing or empty
            if not isinstance(tool_name, str) or not tool_name.strip():
                continue  # skip silently

            tool_name = tool_name.strip()

            args = getattr(item.raw_item, "arguments", None)
            args_str = ""

            if args:
                try:
                    parsed_args = json.loads(args)
                    if parsed_args:
                        args_str = json.dumps(parsed_args)
                except Exception:
                    if args.strip() and args.strip() != "{}":
                        args_str = args.strip()

            args_display = f" with args {args_str}" if args_str else ""

            print(f"{count}. [{agent_name}] → Tool Call: {tool_name}{args_display}")
            count += 1

        elif item.type == "message_output_item":
            print(f"{count}. [{agent_name}] → Message Output")
            count += 1

        else:
            print(f"{count}. [{agent_name}] → {item.type}")
            count += 1

# Example usage:
parse_agent_interaction_flow(result)


## 引用

以下は、最終出力に関連するURL引用を抽出して出力するPythonスニペットです：

In [ ]:
def print_final_output_citations(stream, preceding_chars=50):
    # Iterate over new_items in reverse to find the last message_output_item(s)
    for item in reversed(stream.new_items):
        if item.type == "message_output_item":
            for content in getattr(item.raw_item, 'content', []):
                if not hasattr(content, 'annotations') or not hasattr(content, 'text'):
                    continue
                text = content.text
                for ann in content.annotations:
                    if getattr(ann, 'type', None) == 'url_citation':
                        title = getattr(ann, 'title', '<no title>')
                        url = getattr(ann, 'url', '<no url>')
                        start = getattr(ann, 'start_index', None)
                        end = getattr(ann, 'end_index', None)

                        if start is not None and end is not None and isinstance(text, str):
                            # Calculate preceding snippet start index safely
                            pre_start = max(0, start - preceding_chars)
                            preceding_text = text[pre_start:start].replace('\n', ' ').strip()
                            excerpt = text[start:end].replace('\n', ' ').strip()
                            print("# --------")
                            print("# MCP CITATION SAMPLE:")
                            print(f"#   Title:       {title}")
                            print(f"#   URL:         {url}")
                            print(f"#   Location:    chars {start}–{end}")
                            print(f"#   Preceding:   '{preceding_text}'")
                            print(f"#   Excerpt:     '{excerpt}'\n")
                        else:
                            # fallback if no indices available
                            print(f"- {title}: {url}")
            break

# Usage
print_final_output_citations(result)


In [ ]:
## Deep Research Research Report

print(result.final_output)

### 結論

このノートブックのパターンを使用することで、OpenAI Deep Research Agentsを使用したスケーラブルで本番環境対応の研究ワークフローを構築するための基盤を手に入れることができます。これらの例は、マルチエージェントパイプラインの調整と研究進捗のストリーミング方法だけでなく、外部知識アクセスのためのWeb検索とMCPの統合方法も実証しています。

エージェント型ワークフローを活用することで、単純なQ&Aを超えて、計画、統合、ツール使用を必要とする複雑で多段階の研究タスクに取り組むことができます。モジュラーなマルチエージェント設計（トリアージ、明確化、指示、研究エージェント）により、これらのパイプラインをヘルスケアや金融から技術的デューデリジェンス、市場分析まで、幅広いドメインとユースケースに適応させることができます。

Deep Research APIとAgents SDKが進化し続ける中、これらのパターンは自動化されたデータに基づく研究の最前線に留まるのに役立ちます。内部知識ツールの構築、競合インテリジェンスの自動化、専門アナリストのサポートなど、どのような用途であっても、これらのワークフローは強力で拡張可能な出発点を提供します。

**楽しい研究を！**